# OTP vs. Load Factor Analysis: Southwest's Top 5 Routes

This notebook analyzes the relationship between **Southwest Airlines' On-Time Performance (OTP)** and **Load Factor** on the top 5 domestic routes.

**Research Question:**  
How does Southwest's OTP relate to the Load Factor of its flights on the top 5 domestic routes?

---

## Import Libraries and Setup


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sys
sys.path.append('..')

from src import MetricsCalculator

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# %matplotlib inline